In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import RegularGridInterpolator

from pymedphys.dcm import coords_and_dose_from_dcm
from pymedphys.gamma import gamma_dcm, determine_gamma_inputs

In [ ]:
# x and y labels within this notebook are likely wrong

In [ ]:
lower_percent_dose_cutoff = 20

In [ ]:
gamma_options = {
    'dcm_ref_filepath': "../../../../Downloads/RD.Ecl_WOmask.dcm",
    'dcm_eval_filepath': "../../../../Downloads/RD.Pin.dcm",
    'dose_percent_threshold': 3,
    'distance_mm_threshold': 3,
    'interp_fraction': 10,
    'max_gamma': 1.1,
    'lower_percent_dose_cutoff': lower_percent_dose_cutoff,
    'skip_once_passed': True
}

In [ ]:
gamma = gamma_dcm(**gamma_options)

valid_gamma = gamma[np.invert(np.isnan(gamma))]

100*np.sum(valid_gamma <= 1) / len(valid_gamma)

In [ ]:
gamma_options = {
    'dcm_ref_filepath': "../../../../Downloads/RD.Ecl_WOmask.dcm",
    'dcm_eval_filepath': "../../../../Downloads/RD.Pin.dcm",
    'dose_percent_threshold': 3,
    'distance_mm_threshold': 3,
    'interp_fraction': 10,
    'max_gamma': 1.1,
    'lower_percent_dose_cutoff': lower_percent_dose_cutoff,
    'skip_once_passed': True,
    'local_gamma': True
}


gamma = gamma_dcm(**gamma_options)

valid_gamma = gamma[np.invert(np.isnan(gamma))]

100*np.sum(valid_gamma <= 1) / len(valid_gamma)

In [ ]:
kwargs = determine_gamma_inputs(**gamma_options)

coords_reference = kwargs['coords_reference']
coords_evaluation = kwargs['coords_evaluation']

# coords_reference = coords_reference[1], coords_reference[0], coords_reference[2]
# coords_evaluation = coords_evaluation[1], coords_evaluation[0], coords_evaluation[2]

x_reference, y_reference, z_reference = coords_reference
x_evaluation, y_evaluation, z_evaluation = coords_evaluation

dose_reference = kwargs['dose_reference']
dose_evaluation = kwargs['dose_evaluation']
lower_dose_cutoff = kwargs['lower_dose_cutoff']

In [ ]:
reference_interpolation = RegularGridInterpolator(
    coords_reference, np.array(dose_reference),
    bounds_error=False, fill_value=np.inf
)

In [ ]:
xx, yy = np.meshgrid(x_evaluation, y_evaluation)

xx_flat = np.ravel(xx)
yy_flat = np.ravel(yy)

In [ ]:
lower_dose_cutoff

In [ ]:
relevant_slice = (
    np.max(dose_evaluation, axis=(0, 1)) > 
    lower_dose_cutoff)
slice_start = np.max([
        np.where(relevant_slice)[0][0], 
        0])
slice_end = np.min([
        np.where(relevant_slice)[0][-1], 
        len(z_evaluation)])

In [ ]:
max_ref_dose = np.max(dose_reference)

cut_off_gamma = gamma.copy()
greater_than_2_ref = (cut_off_gamma > 2) & ~np.isnan(cut_off_gamma)
cut_off_gamma[greater_than_2_ref] = 2

for z_i in z_evaluation[slice_start:slice_end:1]:
    i = np.where(z_i == z_evaluation)[0][0]
    
    reshaped_interpolatied_reference_dose = np.reshape(
        reference_interpolation(np.array([xx_flat, yy_flat, np.ones_like(xx_flat) * z_i]).T),
        np.shape(xx)
    )

    reshaped_interpolatied_reference_dose[reshaped_interpolatied_reference_dose == np.inf] = np.nan
    
    print("======================================================================")
    print("Slice = {0}".format(z_i))  
   
    plt.contourf(
        x_evaluation, y_evaluation, dose_evaluation[:, :, i].T, 100, 
        vmin=0, vmax=max_ref_dose, cmap=plt.get_cmap('viridis'))
    plt.title("Evaluation")
    plt.colorbar()
    plt.show()
    
    plt.contourf(
        x_evaluation, y_evaluation, reshaped_interpolatied_reference_dose, 100, 
        vmin=0, vmax=max_ref_dose, cmap=plt.get_cmap('viridis'))
    plt.title("Reference")  
    plt.colorbar()
    plt.show()
    
    plt.contourf(
        x_evaluation, y_evaluation, cut_off_gamma[:, :, i].T, 100, 
        vmin=0, vmax=2, cmap=plt.get_cmap('coolwarm'))
    plt.title("Gamma")
    plt.colorbar()
    plt.show()
    
    plt.contourf(
        x_evaluation, y_evaluation, dose_evaluation[:, :, i].T - reshaped_interpolatied_reference_dose, 100, 
        vmin=-max_ref_dose, vmax=max_ref_dose, cmap=plt.get_cmap('seismic'))
    plt.title("Dose Diff")
    plt.colorbar()
    plt.show()
    
    print("\n")